# Imports

In [95]:
import pandas as pd
import numpy as np
import re

In [96]:
from nba_api.stats.static import teams as nba_teams_module

### Import Team List for Team IDs

In [23]:
from nba_api.stats.static import teams

# get_teams returns a list of 30 dictionaries, each an NBA team.
nba_teams = teams.get_teams()
print("Number of teams fetched: {}".format(len(nba_teams)))
nba_teams[0:29]
teams = pd.DataFrame(nba_teams)
teams = teams.set_index('id') 

Number of teams fetched: 30


,full_name,abbreviation,nickname,city,state,year_founded
id,,,,,,
1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967


## League Standings

In [24]:
from nba_api.stats.endpoints._base import Endpoint
from nba_api.stats.library.http import NBAStatsHTTP
from nba_api.stats.library.parameters import (
    LeagueID,
    Season,
    SeasonType,
    SeasonNullable,
)
from nba_api.stats.endpoints.leaguestandingsv3 import LeagueStandingsV3

In [102]:
def season_standings(season, season_type): 
    league_standings = LeagueStandingsV3(
        league_id="00",
        season=season,              # Example: "2022-23" for the 2022-2023 season
        season_type=season_type   # Example: "Regular Season", "Playoffs", etc.
    )
    standings_data = league_standings.standings.get_dict()
    standings = pd.DataFrame(standings_data['data'], columns=standings_data['headers'])
    standings.index = np.arange(1, len(standings) + 1)
    standings = standings.drop(columns=['SeasonID','LeagueID', 'TeamSlug', 'LeagueRank'])
    return standings 

In [103]:
league_standings = season_standings('2024-25', 'Regular Season')

In [104]:
league_standings = league_standings.rename(columns={'TeamID': 'id'})

## Standings By Division

In [105]:
def process_league_standings(season, season_type):
    # Step 1: Retrieve and rename columns
    league_standings = season_standings(season, season_type)
    league_standings = league_standings.rename(columns={'TeamID': 'id'})
    
    # Step 2: Group by 'Division' and separate into individual DataFrames
    grouped = league_standings.groupby('Division')
    divisions = {division_name: group for division_name, group in grouped}
    
    # Step 3: Prepare and modify DataFrames for each division
    modified_dfs = []
    for division, df in divisions.items():
        # Create a separator row with the division name
        separator_df = pd.DataFrame([{col: '' for col in df.columns}])
        separator_df.iloc[0, df.columns.get_loc('Division')] = division
        
        # Concatenate the separator row and the division DataFrame
        combined_df = pd.concat([separator_df, df])
        
        # Append the combined DataFrame to the list
        modified_dfs.append(combined_df)
    
    # Step 4: Combine all modified DataFrames
    final_df = pd.concat(modified_dfs, ignore_index=True)
    
    # Step 5: Order by 'Conference' and set specific values
    conference_order = ['East', 'West']
    final_df['Conference'] = pd.Categorical(final_df['Conference'], categories=conference_order, ordered=True)
    
    # Setting specific 'Conference' values based on the provided indices
    final_df.loc[0, 'Conference'] = "East"
    final_df.loc[6, 'Conference'] = "East"
    final_df.loc[12, 'Conference'] = "West"
    final_df.loc[18, 'Conference'] = "West"
    final_df.loc[24, 'Conference'] = "East"
    final_df.loc[30, 'Conference'] = "West"
    
    # Step 6: Sort the DataFrame by 'Conference' and 'Division'
    final_df_sorted = final_df.sort_values(by=['Conference', 'Division'])
    
    return final_df_sorted

In [106]:
NBA_Standings_2024 = process_league_standings('2024-25', 'Regular Season')

In [107]:
NBA_Standings_2024

,id,TeamCity,TeamName,Conference,ConferenceRecord,PlayoffRank,ClinchIndicator,Division,DivisionRecord,DivisionRank,WINS,LOSSES,WinPCT,Record,HOME,ROAD,L10,Last10Home,Last10Road,OT,ThreePTSOrLess,TenPTSOrMore,LongHomeStreak,strLongHomeStreak,LongRoadStreak,strLongRoadStreak,LongWinStreak,LongLossStreak,CurrentHomeStreak,strCurrentHomeStreak,CurrentRoadStreak,strCurrentRoadStreak,CurrentStreak,strCurrentStreak,ConferenceGamesBack,DivisionGamesBack,ClinchedConferenceTitle,ClinchedDivisionTitle,ClinchedPlayoffBirth,ClinchedPlayIn,EliminatedConference,EliminatedDivision,AheadAtHalf,BehindAtHalf,TiedAtHalf,AheadAtThird,BehindAtThird,TiedAtThird,Score100PTS,OppScore100PTS,OppOver500,LeadInFGPCT,LeadInReb,FewerTurnovers,PointsPG,OppPointsPG,DiffPointsPG,vsEast,vsAtlantic,vsCentral,vsSoutheast,vsWest,vsNorthwest,vsPacific,vsSouthwest,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Score_80_Plus,Opp_Score_80_Plus,Score_Below_80,Opp_Score_Below_80,TotalPoints,OppTotalPoints,DiffTotalPoints,LeagueGamesBack,PlayoffSeeding,ClinchedPostSeason
0,,,,East,,,,Atlantic,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1610612738,Boston,Celtics,East,0-0,2,,Atlantic,0-0,1,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
2,1610612751,Brooklyn,Nets,East,0-0,4,,Atlantic,0-0,2,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
3,1610612752,New York,Knicks,East,0-0,11,,Atlantic,0-0,3,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
4,1610612755,Philadelphia,76ers,East,0-0,13,,Atlantic,0-0,4,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
5,1610612761,Toronto,Raptors,East,0-0,14,,Atlantic,0-0,5,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
6,,,,East,,,,Central,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
7,1610612741,Chicago,Bulls,East,0-0,3,,Central,0-0,1,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
8,1610612739,Cleveland,Cavaliers,East,0-0,6,,Central,0-0,2,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
9,1610612765,Detroit,Pistons,East,0-0,7,,Central,0-0,3,0,0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0,W 0,0,W 0,0,0,0,W 0,0,W 0,0,W 0,0.0,0.0,0,0,0,0,0,0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0.0,0.0,0.0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,0-0,None,None,None,None,None,None,None,None,None,None,None,None,0-0,0-0,0-0,0-0,0,0,0,0.0,None,0
